In [ ]:
########## auto login ############
import login
from login import *
import pandas as pd
from loguru import logger
logger.add("./logs/SMA_7_logs_{time}.log")
# auto_login()

In [ ]:
# # #######################
# # # MANUAL LOGIN-1 of 1 #
# # #######################

# import logging
# from kiteconnect import KiteConnect
# logging.basicConfig(level=logging.ERROR)
# import threading
# kite = KiteConnect(api_key="w19o0chuo929jxkp")
# import joblib
# kite = joblib.load('kitefile.p')
# print(kite.login_url())
# mytoken = input('Enter Token Here : ')  
# data = kite.generate_session(mytoken,api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
# kite.set_access_token(data["access_token"])
# import joblib
# joblib.dump(kite,'kitefile.p')
# import act
# from act import *
# api_k = "w19o0chuo929jxkp"  # api_key
# api_s = "gsw8ps17ex7lf3cuji4prfnwb4vlyr4y"  # api_secret
# my_ret_token = get_login(api_k, api_s)
# kws = KiteTicker("w19o0chuo929jxkp", my_ret_token)
# from loguru import logger
# logger.add("./logs/SUPERTREND_logs_{time}.log")

In [ ]:
################ NEW VERSION ######################################
fullquant = 100
symbol_ip = 'USDINR20DECFUT'  # USDINR20DECFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
inst_token = 380675  # 380575(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
order_type = 'MIS'
candle_period = 'minute'
exchange_type = kite.EXCHANGE_CDS
offset_quantity = 0
away_from_circuit = 0.50
###################################################################
#fullquant = 50
# symbol_ip = 'GBPINR20NOVFUT'  # USDINR20OCTFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
# inst_token = 289539  # 690691(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
# order_type = 'NRML'
# candle_period = 'minute'
# exchange_type = kite.EXCHANGE_CDS
# offset_quantity = 0
# away_from_circuit = 0.50
# ###################################################################
sma_period1 = 10
sma_period2 = 20
sma_period3 = 40
sma_period4 = 55
sma_period5 = 81
sma_period6 = 100
sma_period7 = 200
####################################################################

myquote = kite.quote([exchange_type+':'+symbol_ip])
upper_circuit = myquote[exchange_type+':'+symbol_ip]['upper_circuit_limit']
lower_circuit = myquote[exchange_type+':'+symbol_ip]['lower_circuit_limit']

logger.debug("Parameters : ")
logger.debug("fullquant : " + str(fullquant))
logger.debug("symbol_ip : " + str(symbol_ip))
logger.debug("upper_circuit : " + str(upper_circuit))
logger.debug("lower_circuit : " + str(lower_circuit))
logger.debug("inst_token : " + str(inst_token))
logger.debug("order_type : " + str(order_type))

In [ ]:
######## CHECK QTY #############
myquantity = 0
def getquant():
    logger.debug("getquant : ")
    global myquantity, order_type, symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if (allpos[i]['tradingsymbol'] == symbol_ip
                and allpos[i]['product'] == order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))
    logger.debug("My Quantity : " + str(myquantity))
    return myquantity
# getquant()

In [ ]:
############### tesating block SUPER-TREND ########################
import datetime
import pandas_ta as ta
candle_interval=candle_period
def calc_sma():
    old_lst = []
    interval = candle_interval
    todaydt = datetime.date.today()
    hud_ago = todaydt - datetime.timedelta(days=6)
    to_date = datetime.date.isoformat(todaydt)
    from_date = datetime.date.isoformat(hud_ago)
    for i2 in range(1):
        new_lst = kite.historical_data(inst_token,
                                       from_date,
                                       to_date,
                                       interval,
                                       continuous=False)
        old_lst = new_lst + old_lst
        todaydt = todaydt - datetime.timedelta(days=7)
        hud_ago = hud_ago - datetime.timedelta(days=7)
        to_date = datetime.date.isoformat(todaydt)
        from_date = datetime.date.isoformat(hud_ago)
    #         print(len(old_lst))

    mydf = pd.DataFrame(old_lst)
    mydata1 = mydf.ta.sma(length=sma_period1)
    mydata2 = mydf.ta.sma(length=sma_period2)
    mydata3 = mydf.ta.sma(length=sma_period3)
    mydata4 = mydf.ta.sma(length=sma_period4)
    mydata5 = mydf.ta.sma(length=sma_period5)
    mydata6 = mydf.ta.sma(length=sma_period6)
    mydata7 = mydf.ta.sma(length=sma_period7)
    
    sma1 = float(mydata1[-1:])
    sma2 = float(mydata2[-1:])
    sma3 = float(mydata3[-1:])
    sma4 = float(mydata4[-1:])
    sma5 = float(mydata5[-1:])
    sma6 = float(mydata6[-1:])
    sma7 = float(mydata7[-1:])
    
    buy_signal = 0
    sell_signal = 0
    
    if((sma1<sma2) and 
       (sma2<sma3) and 
       (sma3<sma4) and 
       (sma4<sma5) and 
       (sma5<sma6) and 
       (sma6<sma7)):
        sell_signal = 1
        
        
    if((sma1>sma2) and 
       (sma2>sma3) and 
       (sma3>sma4) and 
       (sma4>sma5) and 
       (sma5>sma6) and 
       (sma6>sma7)):
        buy_signal = 1
        
    sqaure_signal = 0
    
#     if((myquantity>0 and (sma2<sma3)) or (myquantity<0 and (sma2>sma3))):
#         sqaure_signal = 1
        
    if((myquantity>0 and (sma1<sma7)) or (myquantity<0 and (sma1>sma7))):
        sqaure_signal = 1
    
    logger.debug("buy_signal : " + str(buy_signal))
    logger.debug("sell_signal : " + str(sell_signal))
    
    return {'buy_signal':buy_signal,'sell_signal':sell_signal,
            'sqaure_signal':sqaure_signal
            }
    
results = calc_sma()
results

In [ ]:
########### place order ###########
stopbuy = False
stopsell = False

placing_order = False
is_up = False
is_down = False

def placeneworder(quantdiff, price_ip):
    '''placeneworder(quantdiff,price_ip)'''
    logger.debug('placeneworder @ (' + str(quantdiff) + ',' + str(price_ip) +')')
    global stopbuy, stopsell, order_type, symbol_ip, exchange_type,placing_order,is_up,is_down
    myquantity=getquant()
    if (quantdiff > 0 and stopbuy == False ):
        if(not (is_up == True and is_down == False)):
            try:
                placing_order = True
                order_id = kite.place_order(
                    tradingsymbol=symbol_ip,
                    exchange=exchange_type,
                    transaction_type=kite.TRANSACTION_TYPE_BUY,
                    quantity=abs(quantdiff),
                    price=price_ip,
                    order_type=kite.ORDER_TYPE_MARKET
                    if price_ip == 0 else kite.ORDER_TYPE_LIMIT,
                    variety=kite.VARIETY_REGULAR,
                    product=kite.PRODUCT_MIS
                    if order_type == 'MIS' else kite.PRODUCT_NRML)
                stopsell = False
                logger.debug('Order Successfully Placed @ ' + str(order_type) +' ' + str(quantdiff) + ' ' + str(price_ip))
                is_up = True
                is_down = False

                getquant()
                placing_order = False
                return True
            except Exception as e:
                stopbuy = True
                print(e)
                logger.debug('Order Rejected For @ ' + str(order_type) + ' ' +str(quantdiff) + ' ' + str(price_ip))
                getquant()
                return False
    if (quantdiff < 0 and stopsell == False):
        if(not (is_up == False and is_down == True)):
            try:
                placing_order = True
                order_id = kite.place_order(
                    tradingsymbol=symbol_ip,
                    exchange=exchange_type,
                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                    quantity=abs(quantdiff),
                    price=price_ip,
                    order_type=kite.ORDER_TYPE_MARKET
                    if price_ip == 0 else kite.ORDER_TYPE_LIMIT,
                    variety=kite.VARIETY_REGULAR,
                    product=kite.PRODUCT_MIS
                    if order_type == 'MIS' else kite.PRODUCT_NRML)
                stopbuy = False
                logger.debug('Order Successfully Placed @ ' + str(order_type) + ' ' + str(quantdiff) + ' ' + str(price_ip))
                is_up = False
                is_down = True
                
                getquant()
                placing_order = False
                return True
            except Exception as e:
                stopsell = True
                print(e)
                logger.debug('Order Rejected For @ ' + str(order_type) + ' ' + str(quantdiff) + ' ' + str(price_ip))
                getquant()
                return False

def ckqnt(orderprice, orderquant):
    logger.debug('ckqnt @ (' + str(orderquant))
    global myquantity,placing_order
    quantdiff = orderquant - myquantity
    logger.debug(str(quantdiff) + ' = ' +str(orderquant) + '-' + str(myquantity))
    is_done = False
    if(placing_order==False):
        is_done = placeneworder(quantdiff, orderprice)
    return is_done

last_super_val= 0
update_status_counter = 0

def calc_and_update():
    global last_super_val,update_status_counter,last_ten
    logger.debug('calc_and_update : ')
    super_val = calc_sma()
    update_status_counter = int(update_status_counter)
    update_status(super_val)

    update_status_counter = update_status_counter+1

In [ ]:
################ update status #################
getquant()

def update_status(super_val):
    global fullquant,myquantity
    myquantity = getquant()
    fullquant = int(fullquant)
    if(super_val['buy_signal'] == 1):
        logger.debug('buy_signal = ' + str(super_val['buy_signal']))
        is_done = ckqnt(0, fullquant+offset_quantity)
        if(is_done):
            logger.debug("Order Executed @ "+ 'super_val[buy_signal] = ' + str(super_val['buy_signal'])+' for quant '+str(fullquant))
        else:
            logger.debug("Order Rejected @ "+ 'super_val[buy_signal] = ' + str(super_val['buy_signal'])+' for quant '+str(fullquant))

    if(super_val['sell_signal'] == 1):
        logger.debug('sell_signal = ' + str(super_val['sell_signal']))
        is_done = ckqnt(0, -fullquant+offset_quantity)
        if(is_done):
            logger.debug("Order Executed @ "+ 'super_val[sell_signal] = ' + str(super_val['sell_signal'])+' for quant '+str(fullquant))
        else:
            logger.debug("Order Rejected @ "+ 'super_val[sell_signal] = ' + str(super_val['sell_signal'])+' for quant '+str(fullquant))

    if(super_val['sqaure_signal'] == 1):
        logger.debug('sqaure_signal = ' + str(super_val['sqaure_signal']))
        is_done = ckqnt(0,offset_quantity)
        if(is_done):
            logger.debug("Order Executed @ "+ 'super_val = ' + str(super_val)+' for quant '+str(fullquant))
        else:
            logger.debug("Order Rejected @ "+ 'super_val = ' + str(super_val)+' for quant '+str(fullquant))
            
# update_status()

In [ ]:
############# start trading ############
import threading
import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")
last_min = 0
def on_ticks(ws, ticks):
    global pivot_price, moving_pivot_on, limits, away_from_circuit, lower_circuit, upper_circuit,last_min
    this_min = datetime.datetime.now().minute
        
    if(((upper_circuit-away_from_circuit)>ticks[0]['last_price']) 
       and ((lower_circuit+away_from_circuit)<ticks[0]['last_price'])):
        if (last_min!=this_min):
            last_min = this_min
            print('in the thread : ')
            x = threading.Thread(target=calc_and_update, args=())
            x.start()

        print('=====================================')
        print('LTP : ' + str(ticks[0]['last_price']))

    else:
        ckqnt(0,0)
        if((upper_circuit-away_from_circuit)<=ticks[0]['last_price']):
            logger.debug('Upper Circuit Hit @ '+str(ticks[0]['last_price']))
        if((lower_circuit+away_from_circuit)>=ticks[0]['last_price']):
            logger.debug('Lower Circuit Hit @ '+str(ticks[0]['last_price']))

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

def on_order_update(ws, data):
    logger.debug('on_order_update : ')
    global baseprice, symbol_ip, myquantity, init_qty, current_point
    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

try:
    kws.connect()
except Exception as e:
    logger.debug('Auto relogging in...')
    auto_login()
    kws.stop()
    kws.close()
    kws.connect()